In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
print(os.getcwd())
os.chdir('../../complete_project/../')
print(os.getcwd())
# Then set up the paths
# import sys

# import os
# os.environ['PYTHONPATH'] = os.getcwd()  # Now points to thesis_code directory
# sys.path.append(os.environ['PYTHONPATH'])
sys.path.append("/home/caspar/thesis_code/CellOracle")
sys.path.append("/home/caspar/thesis_code/complete_project/py files")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/AIFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/baseGRNConstructionFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/oracleInferenceFiles")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/oracleSetup")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/AIFiles/batches")
sys.path.append("/home/caspar/thesis_code/complete_project/py files/AIFiles/single")
# import scanpy as sc
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
import os
# import anndata
# import sys
import logging
import HPO_AULC_phase_2 as tuning_module2 # Import the refactored script
import HPO_AULC_phase_1 as tuning_module # Import the refactored script

from datetime import datetime
# import wandb
# import optuna
# #import modified_celloracle as mco
# from CellOracle import celloracle as co
# 
# import CellOracleSetup as setup_module
# import GRNClusterAnalysis as analysis_module
# import GRNInference as inference_module
# import GRNInferenceTest as inference_test_module

log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
log_filename = os.path.join(log_dir, f"app_{datetime.now().strftime('%Y_%m_%d')}.log")

# Configure the basic logging
logging.basicConfig(
    filename=log_filename,
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

/home/caspar/thesis_code/complete_project/notebooks
/home/caspar/thesis_code


/home/caspar/miniconda3/envs/train_model_gpu/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/caspar/miniconda3/envs/train_model_gpu/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

In [2]:
!nvidia-smi


Fri Jun  6 13:09:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:AC:00.0  On |                  Off |
| 30%   38C    P8             25W /  300W |   14810MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

import os
import torch.nn as nn

import warnings

# Add this at the top of your notebook cell before running training
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")


env_type = "NEW_INPUT"
data_path_new_data = os.path.join('../celloracle_data', "celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap")
save_path = os.path.join("../celloracle_data", "models")
notebook_config = {
    # Paths
    "ORACLE_PATH": os.path.join(data_path_new_data, "ready_oracle.pkl"),
    "MODEL_SAVE_PATH_BASE": os.path.join(save_path, "final_optuna_please"), # Base for HPO trial models
    "MODEL_SAVE_PATH": os.path.join(save_path, datetime.now().strftime('%Y_%m_%d_%H_%M_%S') + "_model_"+"FINAL_MODEL_OPTIMIZED_PARAMS_NEW_INPUT"), # Save model with timestamp
    "LOG_DIR_BASE": os.path.join('../celloracle_data', "final_optuna_please"),         # Base for HPO trial logs
    #set wand id to current timestamp
    "WANDB_RUN_ID": datetime.now().strftime('%Y_%m_%d_%H_%M_%S'),
    "WAND_B_LOG_DIR_BASE": os.path.join('../celloracle_data', "final_optuna_please"), # Base for HPO trial wandb logs
    "TRANSITION_MATRIX": os.path.join("../celloracle_data/transition_matrix", "transition_matrix_int32.pkl"), # Path to transition matrix
    "ORACLE_ADATA_PATH": os.path.join(data_path_new_data, "adata.pkl"),
    "PERTURBABLE_GENES_PATH": os.path.join(data_path_new_data, "perturbable_genes.pkl"),
    "RANDOM_SEED" : 77,


    # Training Setup
    "BATCH_SIZE": 256,            # Smaller batch for potentially faster iteration in notebook
    "TOTAL_TIMESTEPS": 100000,
    "TOTAL_TIMESTEPS_HPO" : 2000000,
    "USE_MASKABLE_PPO": False,    # Explicitly choose (make sure sb3_contrib is installed)
    "VERBOSE": 0,
    "LOG_INTERVAL": 10,
    "RESET_NUM_TIMESTEPS": False,
    "DEVICE": "auto",            # Use GPU if available
    "USE_PROGRESS_BAR": True,    # Show progress in notebook output
    "STEP_SAVE_FREQ":500000,

    # CellOracle Parameters

    # PPO Hyperparameters
    "PPO_N_STEPS": 2048,      # Adjust PPO steps,  number of steps before ppo "learns"
    "PPO_N_EPOCHS": 10,
    "PPO_BATCH_SIZE": 128,
    "GAMMA": 0.974,
    "GAE_LAMBDA": 0.95,
    "CLIP_RANGE":0.128,
    "ENT_COEF": 0.00032,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,

    # Environment Specific

    "ALLOW_GENE_ACTIVATION": True,
    "STANDARD_SD_FACTOR": 1.5,
    "USE_LEARNING_RATE_SCHEDULE": True,
    "USE_SIMILARITY_REWARD": True,
    "USE_PREV_KNOCKOUT":True,
    "ENV_TYPE": "FAST",  #options: FIRST,NEW_INPUT, GENERAL,FAST
    "ADD_NOISE":False,
    "VAL_FRAC":0.015,
    "TEST_FRAC":0.015,




    #ENV HYPERPARAMETERS TO TUNE
    "TARGET_CELLS_PER_PHASE": 6,
    "MAX_STEPS_PER_EPISODE": 20,
    "MAX_STEP_INCREASE_PER_PHASE": 4,
    "MAX_STEPS_FIRST_PHASE":150000,
    "PHASE_STEP_INCREASE": 150000,
    "STEP_PENALTY": -1,
    "GOAL_BONUS": 0,
    "DISTANCE_REWARD_SCALE": 5, #the higher the reward, the more we reward/punish getting closer
    "LEARNING_RATE": 1.66e-4,




    # NN Architecture
    "NET_WIDTH_FIRST_P" : 256,
    "NET_WIDTH_SECOND_P" : 256,
    "NET_WIDTH_FIRST_V" : 256,
    "NET_WIDTH_SECOND_V" : 64,
    "ACTIVATION_FN": "relu",    # Specify activation
    "N_ENVS" : 8,
    "HPO_MODE" : True,
    "STUDY_NAME": "studyName_HPO_AULC_phase_1_with_date_" + datetime.now().strftime('%Y_%m_%d_%H_%M_%S'),
    "STORAGE_NAME": "sqlite:///studyName_HPO_AULC_phase_1_with_date_" + datetime.now().strftime('%Y_%m_%d_%H_%M_%S') + ".db",
    
}
if notebook_config["USE_SIMILARITY_REWARD"]:
    notebook_config["DISTANCE_REWARD_SCALE"] =5
else:

    notebook_config["DISTANCE_REWARD_SCALE"] = 5

trained_model = tuning_module.run_optuna_hpo(notebook_config)


[I 2025-11-05 14:16:39,593] A new study created in RDB with name: studyName_HPO_AULC_phase_1_with_date_2025_11_05_14_16_35


running number of trials: 100

--- Starting Optuna HPO Study: studyName_HPO_AULC_phase_1_with_date_2025_11_05_14_16_35 ---
test
Optuna Callback: Initializing dedicated evaluation environment...
IT IS RELOADED!
EVAL MODE INITIATED

---> Starting Optuna Trial 0 <---
  --- Hyperparameters for this Trial ---
  PPO Agent:
    Learning Rate: 0.000249
    N_Steps: 2048
    N_Epochs: 5
    Clip Range: 0.3364
    Entropy Coef: 0.006530
    Gamma (Discount): 0.9903
  Reward Function:
    Goal Bonus: 0.0000
    Step Penalty: -1.0000
    Distance Reward Scale: 3.1276
  ------------------------------------
gpu available: True


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Configuration: {'ORACLE_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/ready_oracle.pkl', 'MODEL_SAVE_PATH_BASE': '../celloracle_data/models/final_optuna_please', 'MODEL_SAVE_PATH': '../celloracle_data/models/final_optuna_please/trial_0', 'LOG_DIR_BASE': '../celloracle_data/final_optuna_please', 'WANDB_RUN_ID': '2025_11_05_14_16_35', 'WAND_B_LOG_DIR_BASE': '../celloracle_data/final_optuna_please', 'TRANSITION_MATRIX': '../celloracle_data/transition_matrix/transition_matrix_int32.pkl', 'ORACLE_ADATA_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/adata.pkl', 'PERTURBABLE_GENES_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/perturbable_genes.pkl', 'RANDOM_SEED': 77, 'BATCH_SIZE': 256, 'TOTAL_TIMESTEPS': 2000000, 'TOTAL_TIMESTEPS_HPO': 2000000, 'USE_MASKABLE_PPO': False, 'VERBOSE': 0, 'LOG_INTERVAL': 10, 'RESET_NUM_TIMESTEPS': False, 'DEVICE': 'auto',

In [ ]:
#DO PHASE 2

import os
import torch.nn as nn

import warnings

# Add this at the top of your notebook cell before running training
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")

env_type = "NEW_INPUT"
data_path_new_data = os.path.join('../celloracle_data',
                                  "celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap")
save_path = os.path.join("../celloracle_data", "models")
notebook_config = {
    # Paths
    "ORACLE_PATH": os.path.join(data_path_new_data, "ready_oracle.pkl"),
    "MODEL_SAVE_PATH_BASE": os.path.join(save_path, "final_optuna_please"),  # Base for HPO trial models
    "MODEL_SAVE_PATH": os.path.join(save_path, datetime.now().strftime(
        '%Y_%m_%d_%H_%M_%S') + "_model_" + "FINAL_MODEL_OPTIMIZED_PARAMS_NEW_INPUT"),  # Save model with timestamp
    "LOG_DIR_BASE": os.path.join('../celloracle_data', "final_optuna_please"),  # Base for HPO trial logs
    #set wand id to current timestamp
    "WANDB_RUN_ID": datetime.now().strftime('%Y_%m_%d_%H_%M_%S'),
    "WAND_B_LOG_DIR_BASE": os.path.join('../celloracle_data', "final_optuna_please"),
    # Base for HPO trial wandb logs
    "TRANSITION_MATRIX": os.path.join("../celloracle_data/transition_matrix", "transition_matrix_int32.pkl"),
    # Path to transition matrix
    "ORACLE_ADATA_PATH": os.path.join(data_path_new_data, "adata.pkl"),
    "PERTURBABLE_GENES_PATH": os.path.join(data_path_new_data, "perturbable_genes.pkl"),
    "RANDOM_SEED": 77,
    #WARNING THIS NEEDS ADJUSTING POSSIBL LATER!
    "TOP_PARAMS_PKL_PATH" : os.path.join(os.path.join(save_path, "final_optuna_please"), "studyName_HPO_AULC_phase_1_with_date_2025_10_22_22_08_24_top_5_params.pkl"),
 
    # Training Setup
    "BATCH_SIZE": 256,  # Smaller batch for potentially faster iteration in notebook
    "TOTAL_TIMESTEPS": 3000000,
    "TOTAL_TIMESTEPS_HPO": 2000,
    "USE_MASKABLE_PPO": False,  # Explicitly choose (make sure sb3_contrib is installed)
    "VERBOSE": 0,
    "LOG_INTERVAL": 10,
    "RESET_NUM_TIMESTEPS": False,
    "DEVICE": "auto",  # Use GPU if available
    "USE_PROGRESS_BAR": True,  # Show progress in notebook output
    "STEP_SAVE_FREQ": 500000,
    "N_EVAL_EPISODES": 1000,

    # CellOracle Parameters

    # PPO Hyperparameters
    "PPO_N_STEPS": 2048,  # Adjust PPO steps,  number of steps before ppo "learns"
    "PPO_N_EPOCHS": 10,
    "PPO_BATCH_SIZE": 128,
    "GAMMA": 0.974,
    "GAE_LAMBDA": 0.95,
    "CLIP_RANGE": 0.128,
    "ENT_COEF": 0.00032,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,

    # Environment Specific

    "ALLOW_GENE_ACTIVATION": True,
    "STANDARD_SD_FACTOR": 1.5,
    "USE_LEARNING_RATE_SCHEDULE": True,
    "USE_SIMILARITY_REWARD": True,
    "USE_PREV_KNOCKOUT": True,
    "ENV_TYPE": "FAST",  #options: FIRST,NEW_INPUT, GENERAL,FAST
    "ADD_NOISE": False,
    "VAL_FRAC": 0.015,
    "TEST_FRAC": 0.015,

    #ENV HYPERPARAMETERS TO TUNE
    "TARGET_CELLS_PER_PHASE": 6,
    "MAX_STEPS_PER_EPISODE": 20,
    "MAX_STEP_INCREASE_PER_PHASE": 6,
    "MAX_STEPS_FIRST_PHASE": 150000,
    "PHASE_STEP_INCREASE": 125000,
    "STEP_PENALTY": -1,
    "GOAL_BONUS": 0,
    #penalty not for same celltype but for same indice, so the action chosen is so bad that the closest neighbor after the action is the same cell
    "DISTANCE_REWARD_SCALE": 5,  #the higher the reward, the more we reward/punish getting closer
    "LEARNING_RATE": 1.66e-4,

    # NN Architecture
    "NET_WIDTH_FIRST_P": 256,
    "NET_WIDTH_SECOND_P": 256,
    "NET_WIDTH_FIRST_V": 256,
    "NET_WIDTH_SECOND_V": 64,
    "ACTIVATION_FN": "relu",  # Specify activation
    "N_ENVS": 8,
    "HPO_MODE": False,
    "STUDY_NAME": "studyName_HPO_AULC_phase_1_with_date_" + datetime.now().strftime('%Y_%m_%d_%H_%M_%S'),
    "STORAGE_NAME": "sqlite:///studyName_HPO_AULC_phase_1_with_date_" + datetime.now().strftime(
        '%Y_%m_%d_%H_%M_%S') + ".db",

}
if notebook_config["USE_SIMILARITY_REWARD"]:
    notebook_config["DISTANCE_REWARD_SCALE"] = 5
else:

    notebook_config["DISTANCE_REWARD_SCALE"] = 5

trained_model = tuning_module2.run_phase2_validation(notebook_config)


--- Loaded 5 top configurations for validation. ---

--- Starting validation for Rank 1 Config ---


--- Running Full Training with Seed: 77 ---
starting run with config: {'ORACLE_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/ready_oracle.pkl', 'MODEL_SAVE_PATH_BASE': '../celloracle_data/models/final_optuna_please', 'MODEL_SAVE_PATH': '../celloracle_data/models/final_optuna_please/rank_01_seed_77', 'LOG_DIR_BASE': '../celloracle_data/final_optuna_please', 'WANDB_RUN_ID': '2025_10_26_23_04_03', 'WAND_B_LOG_DIR_BASE': '../celloracle_data/final_optuna_please', 'TRANSITION_MATRIX': '../celloracle_data/transition_matrix/transition_matrix_int32.pkl', 'ORACLE_ADATA_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/adata.pkl', 'PERTURBABLE_GENES_PATH': '../celloracle_data/celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap/perturbable_genes.pkl', 'RANDOM_SEED': 77, 'TOP_PARAMS_PKL_PATH': 

In [10]:
#lts see if gpus work

import os
import torch.nn as nn
import AI as ai_module # Import the refactored script
import ParameterTuning as tuning_module # Import the refactored script
import multiprocessing
import time
    
def worker_process(gpu_id, base_config):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    print(f"Worker {gpu_id} started, using GPU {gpu_id}")
    tuning_module.run_optuna_hpo(base_config,gpu_id=gpu_id)
    
def start():
    multiprocessing.set_start_method('spawn', force=True)
    data_path_new_data = os.path.join('../celloracle_data', "celloracle_object/new_promoter_without_mescs_trimmed_test_own_umap")
    base_config = {
        # Paths
        "ORACLE_PATH": os.path.join(data_path_new_data, "ready_oracle.pkl"),
        "MODEL_SAVE_PATH_BASE": os.path.join("../celloracle_data", "optuna_hpo_models2"), # Base for HPO trial models
        "LOG_DIR": os.path.join(data_path_new_data, "logs_notebook_exp1"), # Unique logs
        "LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_logs2"),         # Base for HPO trial logs
        "WAND_B_LOG_DIR_BASE": os.path.join('../celloracle_data', "optuna_hpo_wandb_logs2"), # Base for HPO trial wandb logs
        "HPO_FIXED_MAX_STEPS_PER_EPISODE": 20, 
        "TRANSITION_MATRIX": os.path.join("../celloracle_data/transition_matrix", "transition_matrix.pkl"), # Path to transition matrix
    
    
        "BATCH_SIZE": 256,
        "USE_MASKABLE_PPO": False, 
        "VERBOSE": 0,              
        "LOG_INTERVAL": 10,
        "RESET_NUM_TIMESTEPS": False,
        "DEVICE": "auto",
        "USE_PROGRESS_BAR": False,  
        "STEP_SAVE_FREQ": 1000000000, 
        "PPO_N_STEPS": 512,
        "PPO_N_EPOCHS": 10,
        "GAMMA": 0.99,
        "GAE_LAMBDA": 0.95,
        "CLIP_RANGE": 0.2,
        "ENT_COEF": 0.01,       
        "VF_COEF": 0.5,
        "MAX_GRAD_NORM": 0.5,
        "LEARNING_RATE": 3e-4,  
    
        # Environment Specific 
        "ALLOW_GENE_ACTIVATION": True,
        "STANDARD_SD_FACTOR": 1.5,
        "USE_LEARNING_RATE_SCHEDULE": False, 
        "USE_SIMILARITY_REWARD": True,
        "ENV_TYPE": "FAST",  #options: FIRST,NEW_INPUT, GENERAL,FAST
        "USE_PREV_KNOCKOUT": False, 
    
        # ENV HYPERPARAMETERS (Some will be tuned by Optuna)
        "DIVIDER_OF_TOTAL_STEPS_FOR_CURRICULUM_LEARNING": 20,
        "TARGET_CELLS_PER_PHASE": 14,
        "MAX_STEP_INCREASE_PER_PHASE": 8,
        "MAX_STEPS_FIRST_PHASE": 50000, 
        "PHASE_STEP_INCREASE": 15000,
        "MAX_STEPS_PER_EPISODE": 42,
        "OPTUNA_REPORT_INTERVAL": 1000,
    
        # Other ENV params
        "GENE_ACTIVITY_THRESHOLD": 0.01,
        "TARGET_DISTANCE_THRESHOLD": 0.1,
        "DISTANCE_GAMMA": 0.99,           
        "STEP_PENALTY": -1,
        "GOAL_BONUS": 0,
        "FAIL_PENALTY": -3,
        "SAME_CELL_PENALTY": -0.5,
        "DISTANCE_REWARD_SCALE": 5,
        "PARAMETER_TUNING":True,
        "TOTAL_TIMESTEPS_HPO": 4000000,
        "STUDY_NAME": "ppo_celloracle_hpo_general10",
        "STORAGE_NAME": "sqlite:///ppo_celloracle_hpo_general10.db",
    
        # NN Architecture (Fixed for this example, but can be tuned)
        "PI_ARCH": [512,256],
        "VF_ARCH": [512,256],
        "ACTIVATION_FN": nn.ReLU, # Make sure nn is imported if you use this directly
        
    }
    
    if base_config["USE_SIMILARITY_REWARD"]:
        base_config["DISTANCE_REWARD_SCALE"] = 50
    else:
        base_config["DISTANCE_REWARD_SCALE"] = 5

        
    num_gpus = 2
    processes = []
    for gpu_id in range(num_gpus):
        print("test")
        p = multiprocessing.Process(target=worker_process, args=(gpu_id, base_config))
        processes.append(p)
        p.start()
        time.sleep(5)  # Stagger the start time slightly
        
    for p in processes:
        p.join()
        
if __name__ == '__main__':
    start()


ModuleNotFoundError: No module named 'cupy'